In [13]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

In [14]:
class ParserCBRF:
    
    def __init__(self, url):
        self.url = url

    def start(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'data'})
        rows = table.find_all('tr')[1:]
        
        data = {}
        for row in rows:
            cols = row.find_all('td')
            cols = [col.text.strip() for col in cols]
            iso_date = datetime.strptime(cols[0], '%d.%m.%Y').date().isoformat()
            cols[0] = iso_date
            data[iso_date] = cols[1]

        with open('data_1.json', 'w', encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=2, ensure_ascii=False)

        print("Data written to data_1.json file.")

parser = ParserCBRF("http://cbr.ru/hd_base/mb_nd/mb_nd_weekly/")
parser.start()


Data written to data_1.json file.


In [15]:
class MonetaryData:

    def __init__(self, filename):
        self.filename = filename
        with open(filename, 'r', encoding='utf-8') as infile:
            self.data = json.load(infile)

    def monetary_base_by_date(self, date):
        return self.data.get(date, "Data not found for selected date")
    
    def monetary_base_last(self):
        last_date = max(self.data.keys())
        return self.data[last_date]
    
    def range_dates(self, from_date, to_date):
        filtered_data = {}
        for key, value in self.data.items():
            date_obj = datetime.strptime(key, '%Y-%m-%d').date()
            if from_date <= date_obj <= to_date:
                filtered_data[key] = value
        sorted_data = sorted(filtered_data.items())
        return sorted_data

In [16]:
monetary_data = MonetaryData('data_1.json')
print("Данные за определенную дату: " + monetary_data.monetary_base_by_date('2022-08-19')) # введите дату для проверки


monetary_data = MonetaryData('data_1.json')
print("Данные за последнюю доступную дату: " + monetary_data.monetary_base_last())


monetary_data = MonetaryData('data_1.json') 
range_data = monetary_data.range_dates(datetime(2022, 8, 5).date(), datetime(2023, 3, 22).date()) #выберите диапазон дат
print("Данные за определенный период:") 
for data in range_data: 
    print(data)

Данные за определенную дату: 14 479,0
Данные за последнюю доступную дату: 17 593,6
Данные за определенный период:
('2022-08-05', '14 335,0')
('2022-08-12', '14 431,4')
('2022-08-19', '14 479,0')
('2022-08-26', '14 383,5')
('2022-09-02', '14 420,5')
('2022-09-09', '14 558,6')
('2022-09-16', '14 664,3')
('2022-09-23', '14 899,3')
('2022-09-30', '15 265,4')
('2022-10-07', '15 446,0')
('2022-10-14', '15 619,6')
('2022-10-21', '15 670,0')
('2022-10-28', '15 614,5')
('2022-11-03', '15 714,8')
('2022-11-11', '15 789,7')
('2022-11-18', '15 826,9')
('2022-11-25', '15 725,4')
('2022-12-02', '15 722,7')
('2022-12-09', '15 869,3')
('2022-12-16', '16 030,9')
('2022-12-23', '16 189,1')
('2022-12-30', '16 454,7')
('2023-01-13', '16 393,0')
('2023-01-20', '16 397,6')
('2023-01-27', '16 304,5')
('2023-02-03', '16 348,6')
('2023-02-10', '16 437,2')
('2023-02-17', '16 596,2')
('2023-02-22', '16 696,2')
('2023-03-03', '16 721,0')
('2023-03-10', '16 833,7')
('2023-03-17', '16 938,2')
